In [1]:
from encodec import EncodecModel
from encodec.utils import convert_audio

import torchaudio
import torch
from encodec.compress import compress, decompress, decompress_from_frames

In [2]:
model = EncodecModel.encodec_model_24khz()
model.set_target_bandwidth(6.0)

In [3]:
# Load and pre-process the audio waveform
wav, sr = torchaudio.load("/net/papilio/storage2/yhaoyuan/LAbyLM/audios_trimmed_select/3/94.wav")
wav = wav.unsqueeze(0)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
#print(wav.shape)
#compressed = compress(model, wav, use_lm = False)
# Extract discrete codes from EnCodec
with torch.no_grad():
    encoded_frames = model.encode(wav)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)  # [B, n_q, T]

In [4]:
codes.shape

torch.Size([1, 8, 356])

In [5]:
wav, sr = decompress_from_frames(model, codes)
wav.numpy()

array([[ 0.00029862, -0.000193  ,  0.00019739, ...,  0.00032537,
         0.00033618,  0.00034856]], dtype=float32)

In [6]:
import IPython.display as ipd
display(ipd.Audio(wav, rate = 24000))